In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)

In [0]:
import numpy as np
np.random.seed(7)

In [0]:
from keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [0]:
image_shape = (X_train.shape[1], X_train.shape[2])
input_shape = (X_train.shape[1], X_train.shape[2], 1)
print('input shape -',input_shape)

In [0]:
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

In [0]:
X_train = np.reshape(X_train, (len(X_train), input_shape[0], input_shape[1], input_shape[2]))
X_test = np.reshape(X_test, (len(X_test), input_shape[0], input_shape[1], input_shape[2]))

In [0]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Sequential, Model
#from keras import backend as K

In [0]:
encoder = Sequential(name='encoder')

encoder.add(Conv2D(16, (3, 3), activation='relu', padding='same', input_shape=input_shape))
encoder.add(MaxPooling2D((2, 2), padding='same'))

encoder.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
encoder.add(MaxPooling2D((2, 2), padding='same'))    

encoder.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
encoder.add(MaxPooling2D((2, 2), padding='same'))

encoder.summary()

In [0]:
embedding_shape = (4, 4, 8) 

decoder = Sequential(name='decoder')

decoder.add(Conv2D(8, (3, 3), activation='relu', padding='same', input_shape=embedding_shape))
decoder.add(UpSampling2D((2, 2)))

decoder.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
decoder.add(UpSampling2D((2, 2)))

decoder.add(Conv2D(16, (3, 3), activation='relu'))
decoder.add(UpSampling2D((2, 2)))

decoder.add(Conv2D(1, (3, 3), activation='sigmoid', padding='same'))

decoder.summary()

In [0]:
autoencoder = Sequential(name='autoencoder')
autoencoder.add(encoder)
autoencoder.add(decoder)
autoencoder.summary()

In [0]:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [0]:
epochs = 50
batch_size = 256

In [0]:
history = autoencoder.fit(X_train, X_train,
                          epochs=epochs,
                          batch_size=batch_size,
                          shuffle=True,
                          validation_data=(X_test, X_test))

In [0]:
noise_factor = 0.1
noisy_images = X_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X_test.shape) 
noisy_images = np.clip(noisy_images, 0., 1.)

In [0]:
decoded_images = autoencoder.predict(noisy_images)

In [0]:
import matplotlib.pyplot as plot

def show_images(input_images, processed_images, image_shape, number_of_samples=10): 
  if(processed_images is None):
    number_of_rows = 1 
  else:
    number_of_rows = 2

  plot.figure(figsize=(number_of_samples*number_of_rows, 4))

  for sample_index in range(number_of_samples):
    plot_index = sample_index + 1
    ax = plot.subplot(number_of_rows, number_of_samples, plot_index)
    plot.imshow(input_images[sample_index].reshape(image_shape))
    plot.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    if(processed_images is None):
      continue

    plot_index = sample_index + 1 + number_of_samples
    ax = plot.subplot(number_of_rows, number_of_samples, plot_index)
    plot.imshow(processed_images[sample_index].reshape(image_shape))
    plot.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
  plot.show()

In [0]:
show_images(noisy_images, decoded_images, image_shape)